# 神经网络-优化器

神经网络的优化器就是神经网络的参数优化方法。

一个好的优化器应该具有

1. 收敛迅速，需要的迭代次数少


2. 具有鲁棒性，受局部极值、鞍点的影响小

*鲁棒，robust，意稳健，鲁棒是音译，应该是传统说法*

经典的梯度下降并不够通用，于是人们发明了更多的优化器方法来更好地训练神经网络。

可能这一点很多人接触神经网络不够多，所以感触并不深，但是我在做毕业论文时开始用SGD时训练的网络准确率只有40%（10分类），而换用其他优化器之后准确率则高达90%以上，所以在做毕业论文的同时将各种优化器总结一下。

## SGD

SGD（Stochastic gradient descent）随机梯度下降是在梯度下降基础上发展出来的一种，也叫mini-batch梯度下降，和它相对的是full-batch梯度下降，其中分别在经典机器学习算法中的神经网络和逻辑回归介绍过。

SGD的优化可以描述成如下形式：

$$
\begin{split}
    \Theta_t &= \Theta_{t-1} + \Delta\Theta_t\\
    \Delta\Theta_t &= -\alpha\triangledown_{\Theta_t} J(\Theta_t)
\end{split}
\tag{1}
$$

其中α是学习率。

其基本思想就是每次选取少量数据进行训练，这样对比普通的梯度下降有以下**优势**：

1. 训练速度快

SGD一次迭代只用16-1024个样本（通常是2的幂），比把成百上千万的数据一次传入模型要高效得多。


2. 有效避免鞍点

[鞍点](https://en.wikipedia.org/wiki/Saddle_point)（saddle point）指的是一种在各个方向上导数均为0，但函数在这一点并不是极值的点。譬如$z=x^2-y^2$中的（0，0）。

SGD并不会稳健地像极小值前进，而是总体上曲折地向下。正是因为其小样本的随机性带来了噪声，所以在面对鞍点时，SGD会随机向各个方向发散，继而挣脱鞍点的束缚。

和梯度下降一样，SGD的学习率也是一个重要的参数。但是除此之外，样本量的选取是SGD中比较重要的一个选择，样本量的大小何其随机性成反比。相比1024个样本，每次选取32个样本进行迭代随机性更强，噪声也就更大，前进到最优解的道路也就更曲折。而选取1024个样本则会削弱噪声，面对鞍点时跳出鞍点的能力也就更弱。

当然，SGD也有其**劣势**，那就是应对局部极值的能力较差，与full-batch梯度下降相比并没有显著的强化。

*mini-batch的这种思想其实并不是随着神经网络发展起来才兴起（普遍认为神经网路最近一次兴起是从2012年ImageNet上AlexNet大获成功开始）。在机器学习中有一种在线机器学习（[online machine learning](https://en.wikipedia.org/wiki/Online_machine_learning)）的方法。SGD就属于online machine learning的范畴*


## SGD+Momentum

先前我们讲到，SGD的优化完全依赖于当前批量（batch）的数据，缺少了和其他数据的联系，而且SGD对于局部极值没有抵抗能力，于是便有了加入动量的处理。

加入动量的SGD形式如下:

$$
\begin{split}
    \Theta_t &= \Theta_{t-1} + \Delta\Theta_t\\
    \Delta\Theta_t &= \rho\Delta\Theta_{t-1} -\alpha\triangledown_{\Theta_t} J(\Theta_t)
\end{split}
\tag{2}
$$

其中α是学习率，ρ是动量系数,$\rho \in [0,1]$，$\triangledown_{\Theta_t} J(\Theta_t)$是在$\Theta_t$处的梯度。

动量（Momentum）是一个物理学的概念，一个物体的动量指的是这个物体在它运动方向上保持运动的趋势。而在此处,所谓“动量”就是前一步的梯度。

如果把梯度下降比作下山,每次下山都走能使“海拔”下降最大的一步就是梯度下降，而动量的概念引入了惯性。我们下山的过程不再是走一步停下来找下一步，而是“顺势而为”，不仅有当前的梯度还有之前的“惯性”。而为了理解ρ，我们可以想象把一个球推下山，球在下坡时积聚动量，在途中变得越来越快，ρ可视为空气阻力。

动量的引入也从直观上帮助我们理解SGD+Momentum突破局部极值点的能力。

SGD+Momentum从向量合成的角度来看可以表示如下：

![SGD+Momentum](http://188.131.150.243/src/ML_notes/picture/NN+optimizer/SGD+Momentum.PNG)

## Nesterov (Accelerated) Momentum

其实Momentum不是近些年提出的新方法，而是1964年Polyak提出的优化方法。Nesterov Momentum是2012年提出的一个改进。

如果还拿下山的例子来比喻Nesterov Momentum，那Nesterov就是下山一步之前检查一下是否是“正确”的方向，然后下山的时候向正确的方向进行修正。

Nesterov可以表示为如下形式：

$$
\begin{split}
    \Delta \Theta_{t+1} &= \mu \Delta x_t - \alpha \triangledown J(\Theta_t + \mu \Delta \Theta_t)\\
    \Theta_{t+1} &= \Theta_t + \Delta\Theta_{t+1}\\
\end{split}
\tag{3}
$$

Nesterov也可以通过向量合成来表示：

![Nesterov](http://188.131.150.243/src/ML_notes/picture/NN+optimizer/Nesterov.png)

Nesterov在神经网络中实践可能会有一定的琐碎，因为我们知道，神经网络的梯度计算是基于神经网络的中间计算结果（每一层的激活函数值等）的，因此如果我们想计算$\triangledown J(\Theta_t + \mu \Delta \Theta_{t-1})$那么我们将需要把参数带入进行一次前向传播。这样对于一个N层的神经网络，在一次反向传播更新参数的过程中就要前向传播N次，这样的时间消耗对于我们希望的效果来说是不经济的，于是有了使Nesterov计算简洁的一个变体：

假设$\tilde \Theta_t = \Theta_t + \mu \Delta \Theta_t$，原公式将变成：

$$
\begin{split}
    &\begin{cases}
        \Delta \Theta_{t+1} &= \mu \Delta x_t - \alpha \triangledown J(\Theta_t + \mu \Delta \Theta_t)\\
        \Theta_{t+1} &= \Theta_t + \Delta\Theta_{t+1}\\
    \end{cases}\\
    \\
    \Leftrightarrow
    &\begin{cases}
        \Delta \Theta_{t+1} &= \mu \Delta x_t - \alpha \triangledown J(\Theta_t + \mu \Delta \Theta_t)\\
        \Theta_{t+1} + \mu \Delta \Theta_{t+1} &= \Theta_t  + \mu \Delta \Theta_t - \mu \Delta \Theta_t + (1+\mu)\Delta\Theta_{t+1}\\
    \end{cases}\\
    \\
    \Leftrightarrow
    &\begin{cases}
        \Delta \Theta_{t+1} &= \mu \Delta x_t - \alpha \triangledown J(\tilde \Theta_t)\\
        \tilde \Theta_{t+1} &= \tilde \Theta_t - \mu \Delta \Theta_t + (1+\mu)\Delta\Theta_{t+1}\\
    \end{cases}\\
\end{split}
\tag{4}
$$

这样方程组内的第一行就和梯度下降一样，第二行则是当前次梯度下降和上一次梯度下降的一个加权和。

## Adagrad

对机器学习算法熟悉的同学一看到**ada**大概就能猜出来这个方法是什么样的，ada就是adaptive的简称，也就意味着在训练过程中有一定的自适应性。这种自适应性大多都是针对常见/不常见（模型表现好/表现不好等）的样本，进行不同的处理和变动，以提高对不常见（模型表现不好）的样本模型泛化能力。

adagrad（全称可能是Adaptive Gradient algorithm）就是基于这样一种自适应性的思想的方法。

假设，

$$
g_t = \triangledown J(x_t)
\tag{5}
$$

梯度下降类型的更新方法是：

$$
x_{t+1} = x_t - \alpha * g_t
$$

其中α是学习率，而adagrad的更新方法是：

$$
x_{t+1} = x_t - \frac{\alpha}{\sqrt{\sum_{i=1}^t g_i^2 + 1e-7}}g_t
\tag{6}
$$

可以看到adagrad主要是对学习率进行了一个修改，这样的好处主要在于对出现频率较高的参数更新较慢，对出现频率较低的参数更新较快。

学习率的分母实质上是个和梯度形状相同的矩阵（向量），这里的除法其实是对应的逐元素的除法，假如一个参数$x_{ij}$在梯度中出现次数较多，则累加之后就会使得分母越来越大，对应参数的梯度越来越小。而如果一个参数$x_{mn}$几乎从未出现过，则分母累加的结果也会在较小，梯度更新也较大。

adagrad的启动情况有两种，一种是一直都有梯度，则这种在第一次迭代时梯度为1（假设1e-7足够小不造成影响），在之后不断累加，梯度都小于1，且越来越小。另一种情况是参数梯度一直为0，此时分母的1e-7就避免了0/0的情况，直到出现梯度不为零的情况出现之后则和第一类情况类似了。

adagrad的自适应性固然是好的，但是也有一个致命的缺陷，那就是累加之后导致梯度越来越小更新越来越慢。

## RMSprop

RMSprop（Root Mean Square Propagation）是针对adagrad的累加效应改善后期梯度递减情况的一种方法。

它的参数更新方法如下：

$$
\begin{split}
    g_t &= \triangledown J(x_t)\\
    MeanSquare(x, t) &= \rho * MeanSquare(x, t-1) + (1 - \rho)*g_t^2\\
    x_{t} &= x_{t-1} - \frac{\alpha}{\sqrt{MeanSquare(x, t) + 1e-7}}g_t
\end{split}
\tag{7}
$$

其中ρ是衰减率，α是学习率。

RMSprop相当于对adagrad的梯度累加用窗口法进行缩放，给最近的梯度以大的权值，给久远的梯度以小的权值。

## Adam

Adam（Adaptive Moment Estimation）相当于Momentum+Correction+AdaGrad/RMSprop，公式基本如下：

$$
\begin{split}
    g_t &= \triangledown J(x_t)\\
    m_t &= \beta_1 * m_{t-1} + (1 - \beta_1) * g_t\\
    v_t &= \beta_2 * v_{t-1} + (1 - \beta_2) * g_t^2 \\
    \hat m_t &= \frac{m_t}{1 - \beta_1^t}\\
    \hat v_t &= \frac{v_t}{1 - \beta_2^t}\\
    x_{t+1} &= x_{t} - \frac{\alpha}{\sqrt{\hat v_t + 1e-8}}\hat m_t
\end{split}
\tag{8}
$$

官方建议$\beta_1=0.9$，$\beta_2=0.999$。Adam是当前最普bian也是性能最好的优化器（下面提到的AdaBound号称速度堪比Adam，性能堪比SGD）,人们一般都会将Adam当作默认的优化器。

对于adam的解释有这样的说法：$m_t$、$v_t$是对梯度的一阶、二阶矩估计的近似，$\hat m_t$、$\hat v_t$则是对矩估计校正的结果，可以近似看作是无偏估计。

## AdaBound

这是ICLR 2019的最新收录文章，暂时还没有课程会有针对它的简单概括和讲解，等我有空了好好读论文把这个坑填上。

## 论文

把以上方法的原创论文提供给大家，以便深入学习。

- SGD+Momentum：《Some methods of speeding up the convergence of iteration methods》


- NAG(Nesterov Accelerated Gradient)：《On the importance of initialization and momentum in deep learning》


- Adagrad：《Adaptive Subgradient Methods for Online Learning and Stochastic Optimization》


- RMSprop：《Tieleman, T. and Hinton, G. (2012) Lecture 6.5-rmsprop: Divide the Gradient by a Running Average of its Recent Magnitude. COURSERA: Neural Networks for Machine Learning, 4, 26-30. （Unpublished）》


- Adam：《ADAM: A METHOD FOR STOCHASTIC OPTIMIZATION》


- AdaBound：《ADAPTIVE GRADIENT METHODS WITH DYNAMIC BOUND OF LEARNING RATE》